In [10]:
import pandas as pd
import os
import numpy as np

def process_stock_data(main_data_path):
    # 고정된 파일 경로
    nasdaq_data_path = "./Data/Stock/Nasdaq.csv"
    stock_data_dir = "./Data/Stock"
    result_dir = "./Result"

    # Result 폴더 생성 (존재하지 않으면 생성)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    # 메인 데이터 불러오기
    main_df = pd.read_csv(main_data_path)
    main_df['Predict'] = main_df['Predict'].str.strip()

    # Nasdaq 데이터 불러오기
    nasdaq_df = pd.read_csv(nasdaq_data_path)
    nasdaq_df['Date'] = nasdaq_df['Date'].str.strip()

    # 결과 저장용 데이터프레임 초기화
    result_df = pd.DataFrame(columns=[
        "Date", "Long", "Short", "Long_Short", "Cumulative_Long", 
        "Cumulative_Short", "Cumulative_Long_Short", "Nasdaq", "Cumulative_Nasdaq"
    ])

    # 긍정/부정 심볼 정보 저장용 데이터프레임 초기화
    detail_df = pd.DataFrame(columns=["Date", "Type", "Symbol", "Next_Day_Change"])

    # 초기값 (누적 수익률 계산을 위한 초기값)
    cumulative_long = 1
    cumulative_short = 1
    cumulative_long_short = 1
    cumulative_nasdaq = 1

    for date in main_df['Date'].unique():
        # 해당 날짜의 긍정, 부정 데이터 추출
        date_positive = main_df[(main_df['Date'] == date) & (main_df['Predict'] == 'positive')]
        date_negative = main_df[(main_df['Date'] == date) & (main_df['Predict'] == 'negative')]

        # 긍정 및 부정 누적 수익률 계산
        positive_changes = []
        negative_changes = []

        for Symbol in main_df['Symbol'].unique():
            stock_file = os.path.join(stock_data_dir, f"{Symbol}_Stock.csv")
            
            if os.path.exists(stock_file):
                stock_df = pd.read_csv(stock_file)
                stock_df['Date'] = stock_df['Date'].str.strip()
                
                # 긍정 처리 (Long)
                if Symbol in date_positive['Symbol'].values:
                    positive_change = stock_df[stock_df['Date'] == date]['Next_Day_Change'].mean()
                    if not pd.isna(positive_change) and not np.isinf(positive_change):
                        positive_changes.append((Symbol, positive_change))
                        detail_df = pd.concat([detail_df, pd.DataFrame({
                            "Date": [date], "Type": ["Long"], "Symbol": [Symbol], "Next_Day_Change": [positive_change]
                        })])
                
                # 부정 처리 (Short)
                if Symbol in date_negative['Symbol'].values:
                    negative_change = stock_df[stock_df['Date'] == date]['Next_Day_Change'].mean()
                    if not pd.isna(negative_change) and not np.isinf(negative_change):
                        negative_changes.append((Symbol, negative_change))
                        detail_df = pd.concat([detail_df, pd.DataFrame({
                            "Date": [date], "Type": ["Short"], "Symbol": [Symbol], "Next_Day_Change": [negative_change]
                        })])

        # 긍정 및 부정 평균 계산 (inf 제외)
        if positive_changes:
            valid_positive_changes = [val for _, val in positive_changes if not np.isinf(val)]
            long_daily = sum(valid_positive_changes) / len(valid_positive_changes) if valid_positive_changes else 0
        else:
            long_daily = 0

        if negative_changes:
            valid_negative_changes = [val for _, val in negative_changes if not np.isinf(val)]
            short_daily = -sum(valid_negative_changes) / len(valid_negative_changes) if valid_negative_changes else 0
        else:
            short_daily = 0

        long_short_daily = (long_daily + short_daily) / 2

        # 누적 수익률 계산
        cumulative_long *= (1 + long_daily / 100)
        cumulative_short *= (1 + short_daily / 100)
        cumulative_long_short *= (1 + long_short_daily / 100)

        # Nasdaq 데이터에서 Next_Day_Change 가져오기
        nasdaq_row = nasdaq_df[nasdaq_df['Date'] == date]
        nasdaq_change = nasdaq_row['Next_Day_Change'].values[0] if not nasdaq_row.empty else 0
        nasdaq_change = 0 if pd.isna(nasdaq_change) else nasdaq_change

        # Nasdaq 누적 수익률 계산
        cumulative_nasdaq *= (1 + nasdaq_change / 100)

        # 결과 저장
        result_df = pd.concat([result_df, pd.DataFrame({
            "Date": [date],
            "Long": [long_daily],
            "Short": [short_daily],
            "Long_Short": [long_short_daily],
            "Cumulative_Long": [cumulative_long],
            "Cumulative_Short": [cumulative_short],
            "Cumulative_Long_Short": [cumulative_long_short],
            "Nasdaq": [nasdaq_change],
            "Cumulative_Nasdaq": [cumulative_nasdaq]
        })])

    # 날짜별 하위 폴더 생성 및 결과 저장
    result_subdir = os.path.join(result_dir, "Result")
    detail_subdir = os.path.join(result_dir, "Detail")

    os.makedirs(result_subdir, exist_ok=True)
    os.makedirs(detail_subdir, exist_ok=True)

    # 결과 데이터프레임 정렬 및 저장
    result_df = result_df.sort_values(by="Date").reset_index(drop=True)
    detail_df = detail_df.sort_values(by=["Date", "Type", "Symbol"]).reset_index(drop=True)

    # 결과 파일 저장
    base_filename = os.path.basename(main_data_path).replace(".csv", "")
    output_result_file = os.path.join(result_subdir, f"{base_filename}_Result.csv")
    output_detail_file = os.path.join(detail_subdir, f"{base_filename}_Details.csv")

    result_df.to_csv(output_result_file, index=False, encoding="utf-8-sig")
    detail_df.to_csv(output_detail_file, index=False, encoding="utf-8-sig")

    print(f"결과가 {output_result_file} 및 {output_detail_file}에 저장되었습니다.")
    return result_df, detail_df


In [11]:
import os
import warnings
import pandas as pd
import numpy as np

# FutureWarning 끄기
warnings.simplefilter(action='ignore', category=FutureWarning)
# Sentiment 디렉터리 경로
sentiment_dir = "./Sentiment"

# Sentiment 폴더 내 모든 CSV 파일 처리
for file_name in os.listdir(sentiment_dir):
    if file_name.endswith(".csv"):  # CSV 파일만 처리
        file_path = os.path.join(sentiment_dir, file_name)  # 파일 경로 생성
        try:
            # CSV 파일 읽기
            df = pd.read_csv(file_path, nrows=1)  # 첫 행만 읽어 구조 확인
            if "Predict" in df.columns:  # Predict 열이 있는 파일만 처리
                print(f"Processing {file_path}...")
                process_stock_data(file_path)  # 함수 호출
            else:
                print(f"Skipping {file_path} (no 'Predict' column).")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

Processing ./Sentiment/gemma2_27b_FinBERT-BOOTICL.csv...
결과가 ./Result/Result/gemma2_27b_FinBERT-BOOTICL_Result.csv 및 ./Result/Detail/gemma2_27b_FinBERT-BOOTICL_Details.csv에 저장되었습니다.
Processing ./Sentiment/gemma2_27b_CoT.csv...
결과가 ./Result/Result/gemma2_27b_CoT_Result.csv 및 ./Result/Detail/gemma2_27b_CoT_Details.csv에 저장되었습니다.
Processing ./Sentiment/llama3_2_3b_NOCoT.csv...
결과가 ./Result/Result/llama3_2_3b_NOCoT_Result.csv 및 ./Result/Detail/llama3_2_3b_NOCoT_Details.csv에 저장되었습니다.
Processing ./Sentiment/gemma2_27b_BERT-BOOTICL.csv...
결과가 ./Result/Result/gemma2_27b_BERT-BOOTICL_Result.csv 및 ./Result/Detail/gemma2_27b_BERT-BOOTICL_Details.csv에 저장되었습니다.
Processing ./Sentiment/llama3_1_8b_Bootstrap.csv...
결과가 ./Result/Result/llama3_1_8b_Bootstrap_Result.csv 및 ./Result/Detail/llama3_1_8b_Bootstrap_Details.csv에 저장되었습니다.
Processing ./Sentiment/llama3_2_3b_FinBERT-ICL.csv...
결과가 ./Result/Result/llama3_2_3b_FinBERT-ICL_Result.csv 및 ./Result/Detail/llama3_2_3b_FinBERT-ICL_Details.csv에 저장되었습니다.
Proc

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def save_cumulative_returns_chart(csv_file_path, save_folder="./Result/Chart"):
    # 데이터 로드
    df = pd.read_csv(csv_file_path)
    
    # Date 열을 datetime 형식으로 변환
    df['Date'] = pd.to_datetime(df['Date'])

    # Chart 저장 폴더 생성 (존재하지 않으면 생성)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 저장 파일명 생성
    base_filename = os.path.basename(csv_file_path).replace("_Result.csv", "")
    save_path = os.path.join(save_folder, f"{base_filename}_Chart.png")

    # 그래프 생성
    plt.figure(figsize=(12, 6))

    # 각 누적 수익률을 그래프로 표시
    plt.plot(df['Date'], df['Cumulative_Long'], label='Cumulative Long', marker='o', linestyle='-')
    plt.plot(df['Date'], df['Cumulative_Short'], label='Cumulative Short', marker='o', linestyle='--')
    plt.plot(df['Date'], df['Cumulative_Long_Short'], label='Cumulative Long-Short', marker='o', linestyle='-.')
    plt.plot(df['Date'], df['Cumulative_Nasdaq'], label='Cumulative Nasdaq', marker='o', linestyle=':')

    # 그래프 설정
    plt.title(base_filename, fontsize=16)  # 파일명을 제목으로 설정
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("Cumulative Return", fontsize=14)
    plt.grid(True)
    plt.legend(loc='upper left', fontsize=12)

    # X축 월별 표시
    plt.xticks(df['Date'][::30], df['Date'][::30].dt.strftime('%Y-%m'), rotation=45)

    plt.tight_layout()

    # 그래프 저장
    plt.savefig(save_path, format='png')
    plt.close()

    print(f"차트가 {save_path}에 저장되었습니다.")


In [13]:
import os

# Result 디렉터리 내 모든 CSV 파일 처리
result_dir = "./Result/Result"

for file_name in os.listdir(result_dir):
    if file_name.endswith(".csv"):  # CSV 파일만 처리
        csv_file_path = os.path.join(result_dir, file_name)
        try:
            print(f"Processing {csv_file_path}...")
            save_cumulative_returns_chart(csv_file_path)  # 함수 호출
        except Exception as e:
            print(f"Error processing {csv_file_path}: {e}")

Processing ./Result/Result/llama3_1_8b_RoBERTaFinance-ICL_Result.csv...
차트가 ./Result/Chart/llama3_1_8b_RoBERTaFinance-ICL_Chart.png에 저장되었습니다.
Processing ./Result/Result/mistral_7b_BERT-ICL_Result.csv...
차트가 ./Result/Chart/mistral_7b_BERT-ICL_Chart.png에 저장되었습니다.
Processing ./Result/Result/llama3_1_8b_FinBERT-BOOTICL_Result.csv...
차트가 ./Result/Chart/llama3_1_8b_FinBERT-BOOTICL_Chart.png에 저장되었습니다.
Processing ./Result/Result/gemma2_27b_BERT-ICL_Result.csv...
차트가 ./Result/Chart/gemma2_27b_BERT-ICL_Chart.png에 저장되었습니다.
Processing ./Result/Result/mistral_7b_CoT_Result.csv...
차트가 ./Result/Chart/mistral_7b_CoT_Chart.png에 저장되었습니다.
Processing ./Result/Result/FinBERT_Result.csv...
차트가 ./Result/Chart/FinBERT_Chart.png에 저장되었습니다.
Processing ./Result/Result/BERT_Result.csv...
차트가 ./Result/Chart/BERT_Chart.png에 저장되었습니다.
Processing ./Result/Result/llama3_2_3b_FinBERT-ICL_Result.csv...
차트가 ./Result/Chart/llama3_2_3b_FinBERT-ICL_Chart.png에 저장되었습니다.
Processing ./Result/Result/mistral_7b_BERT-BOOTICL_Result.

In [3]:
import os
import pandas as pd

# 설정: 경로
result_root_folder = "./Result/Result"
output_summary_path = os.path.join("./Result", "Summary_Analysis.csv")

# 결과 저장용 리스트
summary_results = []

# 처리 함수
def analyze_strategy(data, column_name, file_path, strategy_name):
    """
    전략별 분석 수행 함수
    """
    # 해당 전략의 수익률 데이터 추출
    returns = data[column_name].dropna()
    if returns.empty:
        return None  # 데이터가 없는 경우 None 반환

    # 분석 지표 계산
    mean_return = returns.mean()  # 이미 백분율로 저장된 값 사용
    std_dev_return = returns.std()  # 백분율 단위로 처리
    sharpe_ratio = (mean_return / std_dev_return) * (252 ** 0.5) if std_dev_return != 0 else 0

    # 누적 수익률 계산
    cumulative_values = (1 + returns / 100).cumprod()
    peak = cumulative_values.cummax()
    drawdown = (cumulative_values - peak) / peak
    mdd = drawdown.min() * 100  # MDD를 %로 변환
    final_return = (cumulative_values.iloc[-1] - 1) * 100  # Final_Return을 %로 변환

    # 결과 반환
    return {
        'Path': file_path,
        'Strategy': strategy_name,
        'Mean_Return': mean_return,
        'Std_Dev_Return': std_dev_return,
        'Sharpe_Ratio': sharpe_ratio,
        'MDD': mdd,
        'Final_Return': final_return
    }

# 파일 처리 루프
for file_name in os.listdir(result_root_folder):
    if file_name.endswith("_Result.csv"):  # "_Result.csv" 파일만 처리
        file_path = os.path.join(result_root_folder, file_name)

        try:
            # CSV 파일 읽기
            data = pd.read_csv(file_path)

            # Nasdaq 분석
            if 'Nasdaq' in data.columns:
                nasdaq_result = analyze_strategy(data, 'Nasdaq', "Nasdaq", "Nasdaq")
                if nasdaq_result and all(res['Strategy'] != "Nasdaq" for res in summary_results):
                    summary_results.append(nasdaq_result)

            # 각 전략 분석
            strategies = ['Long', 'Short', 'Long_Short']
            for strategy in strategies:
                if strategy in data.columns:
                    strategy_result = analyze_strategy(data, strategy, file_path, strategy)
                    if strategy_result:
                        summary_results.append(strategy_result)

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

# 결과를 DataFrame으로 변환 및 저장
summary_df = pd.DataFrame(summary_results)
if not summary_df.empty:
    summary_df.to_csv(output_summary_path, index=False, encoding='utf-8-sig')
    print(f"결과 요약 파일이 저장되었습니다: {output_summary_path}")
else:
    print("처리할 데이터가 없습니다.") 


결과 요약 파일이 저장되었습니다: ./Result/Summary_Analysis.csv
